In [1]:
import csv
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
import pickle as pkl
import matplotlib.pyplot as plt
import random
from pymer4.models import Lmer as lm
import itertools as it
import warnings

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
warnings.filterwarnings('ignore')

In [2]:
gdp_regression_data = pd.read_csv("../data/regression/gdp_regression_data.csv").dropna().reset_index(drop=True)

In [3]:
model_spec = {
    "covariates" : [
        'fd_humidity_[weight]', 'fd_humidity_[weight]_2', 'fd_humidity_annual_std_[weight]', 
        'fd_humidity_annual_std_[weight]_2', 'fd_precip_[weight]', 'fd_precip_[weight]_2', 
        'fd_precip_[weight]_3', 'fd_precip_annual_std_[weight]', 'fd_precip_daily_std_[weight]', 
        'fd_precip_daily_std_[weight]_2', 'fd_precip_daily_std_[weight]_3', 'fd_temp_annual_std_[weight]', 
        'fd_temp_annual_std_[weight]_2', 'fd_temp_annual_std_[weight]_3', 'fd_temp_daily_std_[weight]', 
        'fd_temp_daily_std_[weight]_2', 'humidity_daily_std_[weight]', 'precip_[weight]', 
        'precip_daily_std_[weight]', 'temp_[weight]', 'temp_[weight]_2', 
        'temp_[weight]_3', 'wildfire', 'wildfire_heat_wave'
    ],
    "fixed_effects" : ["country","year"],
    "incremental_effects" : 3,
    "weights" : "unweighted",
    "target" : "fd_ln_gdp"
}

In [4]:
vars_to_grab = []
for covar in model_spec["covariates"]:
    vars_to_grab.append(covar.replace("[weight]",model_spec["weights"]))
for i in range(model_spec["incremental_effects"]):
    for ie_col in [col for col in gdp_regression_data.columns if col.endswith(f"incremental_effect_{i+1}")]:
        vars_to_grab.append(ie_col)

In [5]:
covariate_data = gdp_regression_data[vars_to_grab].to_numpy()
target_data = gdp_regression_data[model_spec["target"]].to_numpy()

In [65]:
model = lm(model_spec["target"] + " ~ " + " + ".join(vars_to_grab) + " + (1|year) + (1|country)", data=gdp_regression_data)

In [68]:
regression = model.fit()

boundary (singular) fit: see help('isSingular') 

Linear mixed model fit by REML [’lmerMod’]
Formula: fd_ln_gdp~fd_humidity_unweighted+fd_humidity_unweighted_2+fd_humidity_annual_std_unweighted+fd_humidity_annual_std_unweighted_2+fd_precip_unweighted+fd_precip_unweighted_2+fd_precip_unweighted_3+fd_precip_annual_std_unweighted+fd_precip_daily_std_unweighted+fd_precip_daily_std_unweighted_2+fd_precip_daily_std_unweighted_3+fd_temp_annual_std_unweighted+fd_temp_annual_std_unweighted_2+fd_temp_annual_std_unweighted_3+fd_temp_daily_std_unweighted+fd_temp_daily_std_unweighted_2+humidity_daily_std_unweighted+precip_unweighted+precip_daily_std_unweighted+temp_unweighted+temp_unweighted_2+temp_unweighted_3+wildfire+wildfire_heat_wave+ABW_incremental_effect_1+AFG_incremental_effect_1+AGO_incremental_effect_1+ALB_incremental_effect_1+AND_incremental_effect_1+ARG_incremental_effect_1+ARM_incremental_effect_1+ASM_incremental_effect_1+ATG_incremental_effect_1+AUS_incremental_effect_1+AUT_incrementa

In [83]:
%load_ext rpy2.ipython

In [87]:
formula = model_spec["target"] + " ~ " + " + ".join(vars_to_grab) + " + (1|year) + (1|country)"

In [121]:
%%R -i gdp_regression_data -i formula -o cov_mat
library(lme4)
library(merDeriv)
model = lmer(formula, data=gdp_regression_data)
cov_mat <- as.matrix(vcov(model, full=TRUE))

Loading required package: nonnest2
This is nonnest2 0.5-7.
nonnest2 has not been tested with all combinations of supported model classes.
Loading required package: sandwich
Loading required package: lavaan
This is lavaan 0.6-18
lavaan is FREE software! Please report any bugs.
fixed-effect model matrix is rank deficient so dropping 120 columns / coefficients
boundary (singular) fit: see help('isSingular')
Error in solve(crossprod(parts$X, t(invVX))) : 
  Lapack dgecon(): system computationally singular, reciprocal condition number = 2.13909e-41
In addition: Warning messages:
1: Some predictor variables are on very different scales: consider rescaling 
2: Some predictor variables are on very different scales: consider rescaling
Error in solve(crossprod(parts$X, t(invVX))) : 
  Lapack dgecon(): system computationally singular, reciprocal condition number = 2.13909e-41


RInterpreterError: Failed to parse and evaluate line 'library(lme4)\nlibrary(merDeriv)\nmodel = lmer(formula, data=gdp_regression_data)\ncov_mat <- as.matrix(vcov(model, full=TRUE))\n'.
R error message: 'Error in solve(crossprod(parts$X, t(invVX))) : \n  Lapack dgecon(): system computationally singular, reciprocal condition number = 2.13909e-41'
R stdout:
Loading required package: nonnest2
This is nonnest2 0.5-7.
nonnest2 has not been tested with all combinations of supported model classes.
Loading required package: sandwich
Loading required package: lavaan
This is lavaan 0.6-18
lavaan is FREE software! Please report any bugs.
fixed-effect model matrix is rank deficient so dropping 120 columns / coefficients
boundary (singular) fit: see help('isSingular')
Error in solve(crossprod(parts$X, t(invVX))) : 
  Lapack dgecon(): system computationally singular, reciprocal condition number = 2.13909e-41
In addition: Warning messages:
1: Some predictor variables are on very different scales: con

In [ ]:
len(cov_mat)

In [ ]:
# From: https://rdrr.io/cran/lme4/man/predict.merMod.html
# There is no option for computing standard errors of predictions because it is difficult to define an efficient method that incorporates uncertainty in the variance parameters; we recommend bootMer for this task. 